In [1]:
from LMAEmbedding import LMAEmbedding
import torch  
import torch.nn  
import torch.nn as nn
import numpy as np

In [2]:
lsh_size = 10
input_dim = 10
embedding_dim = 10
bits_per_chunk = 4 # controls the probability of collison
chunk_size = 2
num_chunks = embedding_dim / chunk_size
num_rep = 1

_weight = nn.Parameter(torch.from_numpy(-1 + 2 * np.random.binomial(1, 0.5, size=((lsh_size,))).astype(np.float32)))
#_weight = nn.Parameter(torch.from_numpy(np.arange(lsh_size).astype(np.float32)))
emb = LMAEmbedding(input_dim, embedding_dim, chunk_size, bits_per_chunk, num_rep, _weight, seed=1)

[single]Potential Memory Usage:  80 memory available 10.0
[total ]Potential Memory Usage:  80 memory available 10
RandomNumbers:  tensor([2038074743, 1791095845,  946286476, 1857819720,     491263])
LMAEmbedding: input_dim:  10  embedding_dim: 10 rep: 1 chunk_size: 2  bits_per_chunk: 4 lsh_mode: srp  seed: 1  lsh matrix torch.Size([10, 1, 20]) memory_size: 10 array_size: 10.0


In [3]:
emb = emb.to(0)
torch.manual_seed(1)
inp = torch.rand(1,input_dim).to("cuda:0")
#inp[1] = inp[0]
#inp[1][-5:] = 0

In [4]:
inp

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999, 0.3971, 0.7544, 0.5695,
         0.4388]], device='cuda:0')

In [8]:
y = emb(inp)
y

tensor([[ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.]], device='cuda:0',
       grad_fn=<LMAEmbeddingFunctionBackward>)

In [7]:
loss = torch.norm(y)

In [9]:
loss.backward()

In [10]:
_weight.grad

tensor([ 0.0000,  0.6325,  0.6325,  0.3162,  0.3162,  0.3162,  0.3162,  0.3162,
        -0.3162,  0.0000], device='cuda:0')

In [7]:
torch.dot(y[0], y[1]) / (1.0 * y.shape[1])

tensor(1., device='cuda:0', grad_fn=<DivBackward0>)

In [8]:
pc = 1 - torch.acos(torch.dot(inp[0], inp[1]) / torch.sqrt(torch.dot(inp[0], inp[0]) * (torch.dot(inp[1], inp[1]))))/ 3.14
print("pc", pc)
print("collison prob", pc**bits_per_chunk)

pc tensor(0.8549, device='cuda:0')
collison prob tensor(0.5342, device='cuda:0')


In [9]:
out = emb(inp)

In [10]:
print("cosine sim", torch.dot(out[0], out[1]) / embedding_dim)
print("equal %", torch.sum(out[0] == out[1])/(1.0 * embedding_dim))

cosine sim tensor(622390., device='cuda:0', grad_fn=<DivBackward0>)
equal % tensor(0., device='cuda:0')


In [11]:
out[0]

tensor([776., 777., 778., 779., 776., 777., 778., 779., 776., 777., 778., 779.,
        776., 777., 778., 779., 776., 777., 778., 779., 776., 777., 778., 779.,
        776., 777., 778., 779., 776., 777., 778., 779., 776., 777., 778., 779.,
        776., 777., 778., 779., 776., 777., 778., 779., 776., 777., 778., 779.],
       device='cuda:0', grad_fn=<SelectBackward>)

In [12]:
out[1]

tensor([799., 800., 801., 802., 799., 800., 801., 802., 799., 800., 801., 802.,
        799., 800., 801., 802., 799., 800., 801., 802., 799., 800., 801., 802.,
        799., 800., 801., 802., 799., 800., 801., 802., 799., 800., 801., 802.,
        799., 800., 801., 802., 799., 800., 801., 802., 799., 800., 801., 802.],
       device='cuda:0', grad_fn=<SelectBackward>)